# This is an implementation of an inverse kinematics algorithm.

In [1]:
import sys
import os

if 'google.colab' in sys.modules:
  print("You are in colab")
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/Othercomputers/Mi portátil/gcs-motion-planning/jupyter_notebooks')
  !pip install pyngrok drake
else:
  os.chdir(os.path.dirname(os.getcwd()))
  print('You are running local.')

import numpy as np

from pyngrok import ngrok

# Drake Libraries
# Libraries for basic sdf file visualization
from pydrake.common import temp_directory
from pydrake.geometry import SceneGraphConfig, StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.multibody.parsing import Parser
import random
from pydrake.multibody.inverse_kinematics import InverseKinematics

from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    Cylinder,
    DiagramBuilder,
    InverseKinematics,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    RigidTransform,
    Role,
    RollPitchYaw,
    RotationMatrix,
    Solve,
    StartMeshcat,
)

# add parent folders to path
sys.path.append(os.path.abspath('..'))



You are in colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Start the visualizer (only run this once).
meshcat = StartMeshcat()
if 'google.colab' in sys.modules:
  # Create a tunnel to the Meshcat server
  ngrok.set_auth_token("2rdws4UBYTqZ1NiiyEbJcWRnByV_7D9Jg8K3YVyZzbFMN1ysg")
  public_url = ngrok.connect(7000, "http")
  print(f"Meshcat is accessible at: {public_url}")

print(os.getcwd())
base_dir = os.path.dirname(os.getcwd())
print(base_dir)


INFO:drake:Meshcat listening for connections at http://localhost:7000


Meshcat is accessible at: NgrokTunnel: "https://bfd4-35-230-81-180.ngrok-free.app" -> "http://localhost:7000"
/content/drive/Othercomputers/Mi portátil/gcs-motion-planning/jupyter_notebooks
/content/drive/Othercomputers/Mi portátil/gcs-motion-planning


In [3]:
def add_custom_robot(plant,base_dir):
    """
    Adds a custom robot to the plant from an SDF file.

    Args:
        plant: The MultibodyPlant to which the robot will be added.
        sdf_path: The path to the SDF file of the robot.

    Returns:
        The model instance of the added robot.
    """
    sdf_path= os.path.join(base_dir,'common-files','models','KINOVA_GEN3_LITE_PRIMITIVES','kinova_model.sdf')

    if not os.path.exists(sdf_path):
        raise FileNotFoundError(f"SDF file not found at {sdf_path}")

    parser = Parser(plant)
    robot_model = parser.AddModels(sdf_path)
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("BASE"))
    return robot_model[0]

def add_custom_wsg(plant, gripper_path=r"/home/drake-tests/common-files/final_gripper/final_gripper.sdf"):
    """
    Adds a custom gripper to the plant from an SDF file.

    Args:
        plant: The MultibodyPlant to which the gripper will be added.
        gripper_path: The path to the SDF file of the gripper.

    Returns:
        The model instance of the added gripper.
    """

    if not os.path.exists(gripper_path):
        raise FileNotFoundError(f"SDF file not found at {gripper_path}")
    parser = Parser(plant)
    wsg_model = parser.AddModels(gripper_path)
    return wsg_model

def add_table(plant, table_path=r"/home/drake-tests/common-files/scene_objects/table/table_wide.sdf"):
    """
    Adds a table to the plant from an SDF file.

    Args:
        plant: The MultibodyPlant to which the table will be added.
        table_path: The path to the SDF file of the table.

    Returns:
        The model instance of the added table.
    """

    if not os.path.exists(table_path):
        raise FileNotFoundError(f"SDF file not found at {table_path}")
    parser = Parser(plant)
    table_model = parser.AddModels(table_path)
    translation_vector = RigidTransform([0.5, -0.5, 0.0])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("table_body"),translation_vector)
    return table_model

## Run IRIS on manually-specified seeds

In [4]:
def create_scene_and_multibody_plant(plant, builder):
    """
    Adds models to the plant

    Returns:
      A tuple of the instances added to the plant.
    """
    robot_model = add_custom_robot(plant)[0]
    table_model = add_table(plant)[0]
    plant.Finalize()

    return (robot_model, table_model)


In [8]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

robot = add_custom_robot(plant,base_dir)
plant.Finalize()

In [9]:

visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(delete_prefix_initialization_event=False),
)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
q0 = plant.GetPositions(plant_context)

diagram.ForcedPublish(context)

In [7]:
meshcat.Delete()
meshcat.DeleteAddedControls()

In [ ]:
def add_constraints(ik, plant, plant_context, gripper_frame, object_frame, object_pose, object_p_AQ_lower, object_p_AQ_upper):
    """Adds constraints to the IK problem."""
    # Fix object pose
    ik.AddPositionConstraint(
        frameB=object_frame,
        p_BQ=[0, 0, 0],
        frameA=plant.world_frame(),
        p_AQ_lower=object_pose.translation(),
        p_AQ_upper=object_pose.translation(),
    )
    ik.AddOrientationConstraint(
        frameAbar=object_frame,
        R_AbarA=RotationMatrix(),
        frameBbar=plant.world_frame(),
        R_BbarB=object_pose.rotation(),
        theta_bound=0.0,
    )

    # Gripper position constraints
    ik.AddPositionConstraint(
        frameB=gripper_frame,
        p_BQ=[0, 0.0, 0.0],
        frameA=object_frame,
        p_AQ_lower=object_p_AQ_lower,
        p_AQ_upper=object_p_AQ_upper,
    )

    # Desired orientation
    desired_rotation_matrix = RotationMatrix(RollPitchYaw([0, 1.5708, 0]))
    ik.AddOrientationCost(
        frameAbar=plant.world_frame(),
        R_AbarA=desired_rotation_matrix,
        frameBbar=gripper_frame,
        R_BbarB=RotationMatrix(),
        c=100.0,
    )

    ik.AddMinimumDistanceLowerBoundConstraint(0.001, 0.05)